In [1]:
import webuiapi
from PIL import Image, PngImagePlugin
import os

## 初始化api
url可以是本地路径，也可以是网络路径

In [2]:
api = webuiapi.WebUIApi()

url = "127.0.0.1"
api = webuiapi.WebUIApi(host=url, port=7860)

## 方法定义
这里是各种方法的定义，有些在下面并没有用上，但写在这里方便随时查看

In [3]:
# 文本生成图片
def text2img(api,prompt,style):
    result = api.txt2img(prompt=prompt,
                         seed=-1,
                         styles=style,
                         cfg_scale=7,
                         sampler_index='DDIM',
                         steps=30,
                         enable_hr=True,
                         hr_scale=1,
                         hr_upscaler=webuiapi.HiResUpscaler.Latent,
                         hr_second_pass_steps=20,
                         hr_resize_x=512,
                         hr_resize_y=768,
                         denoising_strength=0.4,
                         )
    return result

# 保存图片
def saveImages(result, imageDir,imageName,controlnet=False):
    imageNumber = len(result.images)
    # 如果使用了controlnet，那么最后一张图片是controlnet的结果，不需要保存
    if controlnet:
        imageNumber -= 2
    for i in range(imageNumber):
        image = result.images[i]
        pnginfo = PngImagePlugin.PngInfo()
        pnginfo.add_text("parameters", str(result.parameters))
        imagePath = imageDir + "/" + imageName + f"_{i}" + ".png"
        image.save(imagePath, pnginfo=pnginfo)
        
# 图生图
def img2img(api, prompt,imagePath):
    image = Image.open(imagePath)
    result = api.img2img(images=[image], 
                         prompt=prompt,
                         seed=-1, 
                         cfg_scale=6.5, 
                         denoising_strength=0.6
                         )
    return result

# 内画inpaint
def imgInpaint(api, prompt,negative_prompt,imagePath, maskPath):
    image = Image.open(imagePath)
    mask = Image.open(maskPath)
    result = api.img2img(images=[image],
                         mask_image=mask,
                         inpainting_fill=1,
                         prompt=prompt,
                         negative_prompt=negative_prompt,
                         seed=-1,
                         cfg_scale=5.0,
                         denoising_strength=0.7)
    return result

# 放大图片
def scaleImage(api, imagePath, scale):
    image = Image.open(imagePath)
    result = api.extra_single_image(image=image,
                                    upscaler_1=webuiapi.Upscaler.ESRGAN_4x,
                                    upscaling_resize=scale)
    return result

# 批量放大图片
def scaleImages(api, imageDir, scale):
    images = []
    # 遍历 imageDir,读取其中的png文件，放入images列表
    for file in os.listdir(imageDir):
        if file.endswith(".png"):
            image = Image.open(imageDir + "/" + file)
            images.append(image)

    result = api.extra_batch_images(images=images,
                                    upscaler_1=webuiapi.Upscaler.ESRGAN_4x,
                                    upscaling_resize=scale)
    return result

def changeImageSize(width, height):
    # 最小的宽高为 512
    limited = 512
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False

## 参数设定
把各种参数都集中到了这里，方便手改。
controlnet因为设置的时候必须要有图片，所以这里就不设置了，直接在下面的执行代码里面设置。

In [4]:
# 背景描述词
background = ["sea side","On the grass","In front of waterfall","beside a window"]

# 服装描述词
clothes = ["chinese dress",  "wedding dress", "nurse uniform","school uniform"]

# 原图文件夹
originalDir = "E:\SharedDirectory\PoleDriver\AI参考"

# 设置图片保存路径
imageDir = "output"

# mask文件夹
maskDir = "mask"

# 风格描述词
real_stlye = "masterpiece, best quality, ultra high res, highres, best shadow, physics-based rendering, extremely delicate and beautiful,extremely detailed, amazing,shinning skin,"

# 负面描述词
negative_prompt = "easynegative"

# 设置图片描述词
description = "1 girl,look at viewer,"

# 设置批量生成图片的数量
batch_size = 4

## 执行批量生成
先用background和clothes各种组合生成提示词后缀，以及图片后缀名字。然后用这些后缀名字，遍历原图文件夹，用原图和后缀名字组合成新的图片名字，然后保存到新的文件夹里面。

如果目标文件夹里已经有原图片名开头的图片，就说明已经生成过了，会跳过生成。所以批量生成完以后，可以检测下输出结果，不满意的图片删除，然后再重新生成，程序会自动跳过已经生成的图片。

图片的大小设置为与原图比例接近，但最窄的边是512，如果需要修改这个设定，可以去方法`changeImageSize`里面修改。

这里使用的controlnet是openpose和reference的，前者为了生成人体姿态，后者使生成的图片风格接近原图。

openpose的guidance_end设置为0.3，因为前期确定了人体姿态后，后期就不需要再调整了，所以这里设置的比较小。

reference的guidance_end设置为0.7，后面交给ai去自由发挥。如果设置为1，有时候会导致面部破损。

In [5]:


# 因为window里文件命名不能有:，所以不可以直接用提示词后缀来命名图片
suffixs = []
suffixsName = []
for b in background:
    for c in clothes:
        suffixs.append(f"({b}:1.2)" + "," + f"({c}:1.5)")
        suffixsName.append(b + "_" + c)

# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是png文件，跳过
        if not originalImag.endswith(".png"):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 读取原图，以及其大小
        originalImage = Image.open(originalImagePath)
        originalImageWidth = originalImage.size[0]
        originalImageHeight = originalImage.size[1]
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        
        if checkImageExsit(imageDir, originalImagName):
            print(f"图片{originalImagName}已经存在，跳过")
            continue
        
        print(f"原图名字：{originalImagName}。大小：{originalImageWidth}x{originalImageHeight}")
        width,height = changeImageSize(originalImageWidth, originalImageHeight)
        print(f"转换后大小：{width}x{height}")
        
        # 这下面是设置各种controlnet模块
        
        controlnetUnit0 = webuiapi.ControlNetUnit(input_image=originalImage, 
                                        module='openpose_full', 
                                        model='control_v11p_sd15_openpose [cab727d4]',
                                        resize_mode="Crop and Resize",
                                        lowvram=True,
                                        control_mode=0,
                                        guidance_start= 0.0,
                                        guidance_end= 0.3,
                                        threshold_a=100,
                                        threshold_b=200,                                        
                                        )
        
        controlnetUnit1 = webuiapi.ControlNetUnit(input_image=originalImage,
                                        module='reference_only',
                                        resize_mode="Crop and Resize",
                                        lowvram=True,
                                        control_mode=0,
                                        guidance_start= 0.0,
                                        guidance_end= 0.7,
                                        pixel_perfect=True,
                                        )
        
        for i in range(len(suffixs)):
            suffix = suffixs[i]
            prompt = real_stlye + description + suffix
            suffixName = suffixsName[i]
            # 这里根据使用的controlnet模块，设置suffixName前面的类型名
            imageName = originalImagName + "_" + "reference" + suffixName
            
            if checkImageExsit(imageDir, originalImagName):
                print(f"{imageName}已经存在，跳过")
                continue
            
            print(f"使用的提示词是：{prompt}")
            result = api.txt2img(prompt=prompt, 
                                 negative_prompt=negative_prompt,  
                                 seed=-1,
                                 controlnet_units=[controlnetUnit0,controlnetUnit1],
                                 sampler_index='DPM++ 2M Karras',                                 
                                 steps=35,
                                 cfg_scale=6.5,
                                 width=width,
                                 height=height,
                                 enable_hr=True,                                 
                                 hr_resize_x=width,
                                 hr_resize_y=height,
                                 denoising_strength=0.6,
                                 batch_size=batch_size,  
                                 )
            # 保存图片
            saveImages(result, imageDir,imageName,controlnet=True)     

    
    

原图名字：00002。大小：2400x4248
转换后大小：512x896
使用的提示词是：masterpiece, best quality, ultra high res, highres, best shadow, physics-based rendering, extremely delicate and beautiful,extremely detailed, amazing,shinning skin,1 girl,look at viewer,(sea side:1.2),(chinese dress:1.5)
